# Watershed Drop

By Cascade Tuholske 2020.05.21

The goal of this notebook is to drop watersheds that are inland and never reach the ocean. Next I will make a raster with these watersheds to mask out pixels when we calculate zonal stats. 

I am going to find the spatial intersection between EEZs and pour points first.

**Files needed**
- pour points
- inland points
- watersheds 
- eezs

**File to Make** 
- inland watersheds shape file
- coastal watersheds shape file
- inland watersheds raster

In [ ]:
#### Dependencies 
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio 
import matplotlib.pyplot as plt
from shapely.geometry import Point
from multiprocessing import Pool, Queue, Process
import time 
import os
import multiprocessing as mp

In [ ]:
#### Name out put
DATA_IN = '/home/cascade/projects/wastewater/data/'
DATA_OUT = ''

PP_FN = 'raw/pour_points/global_plume_2007_2010.shp'
PP_I_FN = ''
PP_C_FN = ''
WS_FN = ''
WS_I_FN = ''
WS_C_FN = ''
EEZ_FN = 'raw/World_EEZ_v10_20180221/eez_v10.shp'

In [ ]:
#### load files
PP = gpd.read_file(DATA_IN+PP_FN)
# PP_I = gpd.read_file(DATA_IN+PP_I_FN)
# PP_C = gpd.read_file(DATA_IN+PP_C_FN)
# WS = gpd.read_file(DATA_IN+WS_FN)
EEZ = gpd.read_file(DATA_IN+EEZ_FN) # are in epsg: 54009

## Plot them to check them

In [ ]:
print(EEZ.crs)
print(PP.crs)

In [ ]:
### reproject PP ... have to load twice. Do not know why
PP = PP.to_crs({'init': 'epsg:4326'}) # switch crs
PP.drop(['SUM_FERTC','SUM_PESTC', 'SUM_IMPV'], axis= 1, inplace = True)
PP.head()

In [ ]:
PP.plot()

## Buffer and Drop

In [ ]:
def point_buffer(gpd_df, radius):
    """Function to make a shapely polygon buffer around a point. Be sure to check crs to enter radius correctly
    
    Args: gpd_df = geopandas df
          raduis = radius to dialate points
    """

    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [ ]:
#### Find intersection
def intersect(point_buffer, polys):
    
    "finds intersection of buffered points and polys, returns point buffer geom and ids"
    
    basin_id_list = []
    geom_list = []
    
    for index_Pt, point in point_buffer.iterrows():
        for index_poly, poly, in polys.iterrows():
            if point['geometry'].intersects(poly['geometry']):

                # get ID and geom for pour point that is in EEZ
                basin_id = point['basin_id']
                geom = point['geometry']

                # write to list
                basin_id_list.append(basin_id)
                geom_list.append(geom)
                
    # return a data frame 
    df = pd.DataFrame()
    df['basin_id'] = basin_id_list
    df['geometry'] = geom_list
    
    return df

    

In [ ]:
#### Buffer the point by 0.1 degrees 
PP_buffer = point_buffer(PP, radius = 0.1)

In [ ]:
#### Drop columns from EEZ
EEZ_drop = gpd.GeoDataFrame(EEZ[['MRGID', 'geometry']])

In [ ]:
#### Find intersection
PP_coastal = intersect(PP_buffer, EEZ_drop)

In [ ]:
len(PP_coastal)

In [ ]:
len(PP_buffer)

In [ ]:
len(PP_buffer)

In [ ]:
#### merge based on ids